In [1]:
import sbol2
from typing import Dict, List
from sbol2build.abstract_translator import extract_design_parts

In [2]:
abstract_doc = sbol2.Document()
abstract_doc.read("tests/test_files/combinatorial_1.xml")
print(abstract_doc)

Design........................0
Build.........................0
Test..........................0
Analysis......................0
ComponentDefinition...........5
ModuleDefinition..............0
Model.........................0
Sequence......................4
Collection....................0
Activity......................0
Plan..........................0
Agent.........................0
Attachment....................0
CombinatorialDerivation.......1
Implementation................0
SampleRoster..................0
Experiment....................0
ExperimentalData..............0
Annotation Objects............5
---
Total: .........................15



In [3]:
toplevel = abstract_doc.componentDefinitions[
    2
]  # TODO develop approach to extract toplevel definition in non MD documents; maybe check for annotations or subcomponents
print(toplevel)

https://sbolcanvas.org/abstract_combinatorial/1


In [4]:
parts = extract_design_parts(toplevel, abstract_doc)

In [5]:
for part in parts:
    print(part)

https://synbiohub.org/user/Gon/CIDARMoCloParts/J23100/1
https://sbolcanvas.org/RBS_HFQr/1
https://synbiohub.org/user/Gon/CIDARMoCloParts/E0040m_gfp/1
https://synbiohub.org/user/Gon/CIDARMoCloParts/B0015/1


In [6]:
def extract_combinatorial_design_parts(
    design: sbol2.ComponentDefinition, doc: sbol2.Document, plasmid_doc
) -> Dict[str, List[sbol2.ComponentDefinition]]:
    """
    Extracts and returns a mapping of component definitions from a combinatorial design, in order.

    Retrieves the components in sequential order from the given design.
    Variants of combinatinatorial components are entered in a list corresponding to the URI of the component in the abstract design.

    Args:
        design:
            The :class:`sbol2.ComponentDefinition` representing the top-level design
            from which to extract parts.
        doc:
            The primary :class:`sbol2.Document` containing the base component definitions
            and combinatorial derivations.
        plasmid_doc:
            An additional :class:`sbol2.Document` used to resolve component variants
            (plasmid-specific variants referenced by combinatorial derivations).

    Returns:
        Dict[str, List[sbol2.ComponentDefinition]]:
            A dictionary mapping component identities to lists
            of corresponding component definitions.

            - Sequential design components map to lists containing a single definition.
            - Combinatorial variable components map to lists of variant definitions.
    """
    component_list = [c for c in design.getInSequentialOrder()]
    component_dict = {
        component.identity: [doc.getComponentDefinition(component.definition)]
        for component in component_list
    }

    for deriv in doc.combinatorialderivations:
        for component in deriv.variableComponents:
            component_dict[component.variable] = [
                plasmid_doc.getComponentDefinition(var) for var in component.variants
            ]

    return component_dict

### Problem: variant definitions are not included in sbolCanvas abstract combinatorial design document, only references
#### Solution: references can be resolved by pulling from plasmid collection

In [7]:
plasmid_collection = sbol2.Document()
sbh = sbol2.PartShop("https://synbiohub.org")
sbh.pull(
    "https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/CIDARMoCloPlasmidsKit_collection/1/b7fdc21c6601a61d3166073a9e50f2c3843e1df5/share",
    plasmid_collection,
)

In [8]:
dict = extract_combinatorial_design_parts(toplevel, abstract_doc, plasmid_collection)

for key, value in dict.items():
    print(f"{key}:")
    for element in value:
        print(element.displayId)

https://sbolcanvas.org/abstract_combinatorial/J23100_1/1:
J23100
https://sbolcanvas.org/abstract_combinatorial/RBS_HFQr_2/1:
B0032
B0033
https://sbolcanvas.org/abstract_combinatorial/E0040m_gfp_3/1:
E0040m_gfp
https://sbolcanvas.org/abstract_combinatorial/B0015_4/1:
B0015


In [9]:
import itertools


def enumerate_design_variants(component_dict):
    """
    Given a dict mapping variable component identities to lists of ComponentDefinitions,
    generate all possible design combinations as lists of ComponentDefinitions
    (in consistent order of keys).
    """
    keys = list(component_dict.keys())
    variant_lists = [component_dict[k] for k in keys]

    # Cartesian product across all variant lists
    all_variants = list(itertools.product(*variant_lists))

    # Convert tuples to lists (optional) for clarity
    all_variants = [list(combo) for combo in all_variants]

    return all_variants

In [10]:
from sbol2build.abstract_translator import (
    construct_plasmid_dict,
    extract_toplevel_definition,
    get_compatible_plasmids,
    MocloPlasmid,
)

enumerated = enumerate_design_variants(dict)

plasmid_dicts = []

# for design in enumerated:
#     plasmid_dicts.append(construct_plasmid_dict(design, plasmid_collection))

In [18]:
def translate_abstract_to_plasmids(
    abstract_design_doc: sbol2.Document,
    plasmid_collection: sbol2.Document,
    backbone_doc: sbol2.Document,
):
    backbone_def = extract_toplevel_definition(backbone_doc)
    backbone_plasmid = MocloPlasmid(backbone_def.displayId, backbone_def, backbone_doc)

    # combinatorial design
    if len(abstract_design_doc.combinatorialderivations) > 0:
        abstract_design_def = abstract_design_doc.getComponentDefinition(
            abstract_design_doc.combinatorialderivations[0].masterTemplate
        )

        combinatorial_part_dict = extract_combinatorial_design_parts(
            abstract_design_def, abstract_design_doc, plasmid_collection
        )
        enumerated_part_list = enumerate_design_variants(combinatorial_part_dict)

        final_plasmid_list = []

        for design in enumerated_part_list:
            plasmid_dict = construct_plasmid_dict(design, plasmid_collection)
            final_plasmid_list += get_compatible_plasmids(
                plasmid_dict, backbone_plasmid
            )

        return set(final_plasmid_list)

    # generic design
    else:
        abstract_design_def = extract_toplevel_definition(abstract_design_doc)

        ordered_part_definitions = extract_design_parts(
            abstract_design_def, abstract_design_doc
        )

        plasmid_dict = construct_plasmid_dict(
            ordered_part_definitions, plasmid_collection
        )

        return get_compatible_plasmids(plasmid_dict, backbone_plasmid)

In [19]:
bb_doc = sbol2.Document()
sbh.pull(
    "https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/DVK_AE/1/647c5b2458567dcce6b0a37178d352b8ffa9a7fe/share",
    bb_doc,
)

translate_abstract_to_plasmids(
    abstract_design_doc=abstract_doc,
    plasmid_collection=plasmid_collection,
    backbone_doc=bb_doc,
)

found: https://synbiohub.org/user/Gon/CIDARMoCloParts/J23100/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_AB/1 with ['Fusion_Site_B', 'Fusion_Site_A']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/J23100/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_GB/1 with ['Fusion_Site_B', 'Fusion_Site_G']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/J23100/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_FB/1 with ['Fusion_Site_F', 'Fusion_Site_B']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/J23100/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_EB/1 with ['Fusion_Site_B', 'Fusion_Site_E']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/B0032/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pB0032_BC/1 with ['Fusion_Site_C', 'Fusion_Site_B']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/E0040m_gfp/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pE0040_CD/1 with ['

{MocloPlasmid:
   Name: B0015_D_E
   Definition: https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pB0015_DE/1
   Fusion Sites: ['D', 'E'],
 MocloPlasmid:
   Name: B0032_B_C
   Definition: https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pB0032_BC/1
   Fusion Sites: ['B', 'C'],
 MocloPlasmid:
   Name: B0033_B_C
   Definition: https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pB0033_BC/1
   Fusion Sites: ['B', 'C'],
 MocloPlasmid:
   Name: E0040m_gfp_C_D
   Definition: https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pE0040_CD/1
   Fusion Sites: ['C', 'D'],
 MocloPlasmid:
   Name: J23100_A_B
   Definition: https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_AB/1
   Fusion Sites: ['A', 'B']}